In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from collections import OrderedDict
import json

In [2]:
path_rmqs = '/home/administrateur/Bureau/Datagrosyst/data_entrepot_&_outils/referentiels/'

rmqs1 = pd.read_csv(path_rmqs + 'Raw_data_rmqs_1.csv')[['id_site','y_theo','x_theo','sampling_date']]
rmqs2 = pd.read_csv(path_rmqs + 'Raw_data_rmqs_2.csv')[['id_site','y_theo','x_theo','sampling_date']]

rmqs = pd.concat([rmqs1,rmqs2])
del(rmqs1, rmqs2)


def get_unique_txt(series):
    if len(series.dropna().unique()) == 0:
        return np.nan
    elif len(series.dropna().unique()) == 1:
        return series.dropna().unique().item()
    elif len(series.dropna().unique()) > 1:
        return list(series.dropna().unique())

agg_dict = {
    'y_theo' : get_unique_txt,
    'x_theo' : get_unique_txt,
    'sampling_date' : get_unique_txt
}

rmqs_end = rmqs.groupby('id_site').agg(agg_dict).reset_index()
rmqs_end = rmqs_end.rename(columns={'x_theo' : 'longitude',
                                    'y_theo' : 'latitude'})

rmqs_end.to_csv(path_rmqs + 'rmqs_site_id.csv', index=False)

In [ ]:
li = []
for idx, row in rmqs_end.iterrows():
    d = OrderedDict()
    d['type'] = 'Feature'
    d['geometry'] = {
        'type': 'Point',
        'coordinates': [row['longitude'], row['latitude']]
    }
    d['properties'] = {
        'id_site' : row['id_site'],
        'sampling_date' : row['sampling_date']
    }
    li.append(d)

d = OrderedDict()
d['type'] = 'FeatureCollection'
d['features'] = li

with open(path_rmqs + 'rmqs_site_id.json','w') as f:
    json.dump(d,f,indent=2)

rmqs = gpd.read_file(path_rmqs + 'rmqs_site_id' + '.json').set_crs(2154, allow_override=True)
rmqs = rmqs.to_crs(4326)

with open(path_rmqs + 'rmqs_site_id.json','w') as f:
    f.write(rmqs.to_json())

EPSG:2154


In [ ]:
path_geoVec = '/home/administrateur/Bureau/Datagrosyst/data_entrepot_&_outils/referentiels/geospatial_data/'

# Attention on passe en projection. On reviendra en 4326 après le sjoin_nearest
com = gpd.read_file(path_geoVec + 'geoVec_com2024' + '.json').to_crs(3857)
metrocom = com[~(com['dep'].str.match(r'97+') | com['dep'].str.match(r'98+'))]
metrocom = metrocom[['codgeo','geometry']].rename(columns={'codgeo': 'codeinsee'})
metrocom = metrocom.assign(centroid = metrocom.centroid)

com_rmqs = gpd.sjoin_nearest(metrocom, rmqs, distance_col="distances", how='left')[['codeinsee','id_site','sampling_date','distances']].set_index('codeinsee')
com_rmqs['distances'] = round(com_rmqs['distances']/1000, 1)
com_rmqs = com_rmqs.rename(columns={
    'id_site' : 'rmqs_site_id',
    'sampling_date' : 'rmqs_date_sampl',
    'distances' : 'rmqs_dist_site'
})

# rmqs_site_id = identifiant du site RMS le plus proche du centroide de la commune métropolitaine
# rmqs_date_sampl = date d'échantillonnage sur le site RMQS en question (attention 2 campagnes distinctes)
# rmqs_dist_site = distances entre le centroide de la commune métropolitaine la plus proche du site RMQS indiqué et le point du site RMQS indiqué